In [ ]:
import cv2 #importing open cv
import numpy as np #importing numpy

yolo = cv2.dnn.readNet("yolov-custom.cfg","C:\\Users\\abhishek sai\\Documents\\yolov-custom.weights")#To load the Deep Learning network to the system when the OpenCV library is used.
classes = [] #list named classes

with open("coco.names", "r") as file: #opening the file with read permssion
    classes = [line.strip() for line in file.readlines()] #to read the file line by line and remove the spaces
layer_names = yolo.getLayerNames()# It gives you list of all layers used in a network.
output_layers = [layer_names[i[0] - 1] for i in yolo.getUnconnectedOutLayers()]# It gives you list of all layers used in a network.
#To get the corresponding indexes, we need to do layer_names[i[0] - 1]. 

# to draw rectangles
colorRed = (0,0,255)#assigning the colour red
colorGreen = (0,255,0)#assigning the colour green

# #Loading Images
name = "C:\\Users\\abhishek sai\\Downloads\\1.jpg"#path
img = cv2.imread(name)#reading image
height, width, channels = img.shape  #specifying dimensions because the network contain height and width

# Detecting objects
# remove intensity ( noise ) it brings to same level in which yolo is trained ir bringig to original
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False) #detection of objects done from the cv2 

yolo.setInput(blob)#setting the imput
# read the yolo network
outputs = yolo.forward(output_layers)#Runs a forward pass to compute the net output. your detection i.e net. forward() will give Numpy ndarray as output which you can use it to plot box on the given input image.

#declaring lists for future usage
class_ids = []  # labels
confidences = [] # remove noise
boxes = [] # detected objects 
for output in outputs: #using nested loops to go through the image and dectection
    for detection in output:
        scores = detection[5:]  # identify score which is important 
        class_id = np.argmax(scores) # take max of score
        confidence = scores[class_id] 
        if confidence > 0.5: # take more than 50%
            center_x = int(detection[0] * width) # where the objects are 
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)  
            h = int(detection[3] * height)

            x = int(center_x - w / 2) # we move it to top left corner
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h]) # coordinates of object
            confidences.append(float(confidence))
            class_ids.append(class_id)

# it detects the error so it removes
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)#NMSBoxes used to suppress weak, overlapping bounding boxes.
for i in range(len(boxes)):#For loop for drawing rectangles and labling those boxes
    if i in indexes:
        x, y, w, h = boxes[i] # exctracting coordinates 
        label = str(classes[class_ids[i]])
        cv2.rectangle(img, (x, y), (x + w, y + h), colorGreen, 3)#for rectangle
        cv2.putText(img, label, (x, y + 10), cv2.FONT_HERSHEY_PLAIN, 8, colorRed, 8)#for text

#This shows the output image with the object identification boxes with lables
cv2.imshow("Image", img)
cv2.imwrite("output.jpg",img)
cv2.waitKey(0)